# Assignment Chapter 2 - DEEP LEARNING [Case #4]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Task: **CLASSIFICATION**
* DL Framework: **PyTorch**
* Dataset: Credit Card Fraud 2023
* Libraries: Pandas/cuDF, Scikit-learn/cuML, Numpy/cuPy
* Objective: Classify credit fraud transactions using Multilayer Perceptron

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu mengklasifikasi transaksi fraud menggunakan *Multilayer Perceptron*

### Import Libraries

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 24.6.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com

        ***********************************************************************
        The pip install of RAPIDS is complete.
        
        Please do not run any further installation from the conda based installation methods, as they may cause issues!
        
        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files. 
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        *****************************************************************

In [ ]:
import shutil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

<font color="red">**- - - - MOHON DIPERHATIKAN - - - -**</font>
<br>**Aktifkan GPU sekarang.** Di Google Colab, klik **Runtime > Change Runtime Type**, lalu pilih **T4 GPU**.

### Dataset Loading (CPU vs. GPU)

In [ ]:
from pandas import read_csv as read_by_CPU
from cudf import read_csv as read_by_GPU

In [ ]:
# unzip the file
shutil.unpack_archive('dataset_case_04.zip', '/content/sample_data/', 'zip')

In [ ]:
# TODO: Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
%time data_cpu = read_by_CPU('/content/sample_data/creditcard_2023.csv')

CPU times: user 6.62 s, sys: 300 ms, total: 6.92 s
Wall time: 12 s


In [ ]:
# Impor dataset dengan cuDF (Pandas di GPU)
%time data_gpu = read_by_GPU('/content/sample_data/creditcard_2023.csv')

CPU times: user 342 ms, sys: 351 ms, total: 694 ms
Wall time: 1.12 s


In [ ]:
# TODO: Hilangkan kolom ID
data_gpu = data_gpu.drop('id', axis=1)
data_cpu = data_cpu.drop('id', axis=1)

### Standardization (CPU vs. GPU)

In [ ]:
from sklearn.preprocessing import StandardScaler as StandardScalerCPU
from cuml.preprocessing import StandardScaler as StandardScalerGPU

In [ ]:
ScalerCPU = StandardScalerCPU()
ScalerGPU = StandardScalerGPU()

arbitrary_features = ["V"+str(i+1) for i in range(27)]

In [ ]:
%%time

data_cpu[arbitrary_features] = ScalerCPU.fit_transform(data_cpu[arbitrary_features].values)
data_cpu["Amount"] = ScalerCPU.fit_transform(data_cpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 272 ms, sys: 613 ms, total: 885 ms
Wall time: 854 ms


In [ ]:
%%time

data_gpu[arbitrary_features] = ScalerGPU.fit_transform(data_gpu[arbitrary_features].values)
data_gpu["Amount"] = ScalerGPU.fit_transform(data_gpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 658 ms, sys: 457 ms, total: 1.11 s
Wall time: 1.09 s


### Train/Test Split (CPU vs. GPU)

In [ ]:
from sklearn.model_selection import train_test_split as splitCPU
from cuml.preprocessing import train_test_split as splitGPU

In [ ]:
# TODO: Tentukan X (features) dan Y (target), gunakan "data_gpu"
X = data_gpu.drop('Class', axis=1)
Y = data_gpu['Class']

In [ ]:
%%time

# TODO: Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
test_size = 0.2
random_state = 42
x_train, x_test, y_train, y_test = splitCPU(X, Y, test_size=test_size, random_state=random_state)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (454904, 29)
x_test shape:  (113726, 29)
CPU times: user 42.4 ms, sys: 18.2 ms, total: 60.6 ms
Wall time: 59.7 ms


In [ ]:
%%time

# TODO: Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
test_size = 0.2
random_state = 42
x_train, x_test, y_train, y_test = splitGPU(X, Y, test_size=test_size, random_state=random_state)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (454904, 29)
x_test shape:  (113726, 29)
CPU times: user 95 ms, sys: 32.3 ms, total: 127 ms
Wall time: 124 ms


### Convert the dataset into Tensor

In [ ]:
import cupy # Numpy for GPU

In [ ]:
# TODO: Aktifkan GPU (CUDA) sebagai device untuk training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
x_train_tensor = torch.from_numpy(cupy.asnumpy(x_train.values)).to(device)
y_train_tensor = torch.from_numpy(cupy.asnumpy(y_train.values)).to(device)

x_test_tensor = torch.from_numpy(cupy.asnumpy(x_test.values)).to(device)
y_test_tensor = torch.from_numpy(cupy.asnumpy(y_test.values)).to(device)

Train_tensor = TensorDataset(x_train_tensor, y_train_tensor)
Test_tensor = TensorDataset(x_test_tensor, y_test_tensor)

### Batching the Dataset with PyTorch DataLoader

In [ ]:
# TODO: Tentukan nilai batch
batch_size = 64

Train_dataset = DataLoader(Train_tensor, batch_size=batch_size, shuffle=True)
Test_dataset = DataLoader(Test_tensor, batch_size=batch_size, shuffle=False)

### Model Blueprint

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, input_dim, num_neurons):
        super(FeedForward, self).__init__()
        self.input_dim = input_dim
        self.num_neurons = num_neurons

        self.net = nn.Sequential(
            nn.Linear(self.input_dim, self.num_neurons),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

    def to(self, device):
        self.net.to(device)
        return self

class Net(nn.Module):
    def __init__(self, in_features, num_layers, num_neurons):
        super(Net, self).__init__()
        self.in_features = in_features
        self.num_layers = num_layers
        self.num_neurons = num_neurons

        self.fc1 = nn.Linear(self.in_features, self.num_neurons)
        self.relu = nn.ReLU()
        self.blocks = [FeedForward(self.num_neurons, self.num_neurons).to(device) \
                       for _ in range(self.num_layers)]
        self.output_layer = nn.Linear(self.num_neurons, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.relu(self.fc1(x))

        for block in self.blocks:
            output = block(output)
        output = self.sigmoid(self.output_layer(output))

        return output

### Model Hyperparameters and Parameters

In [ ]:
# [ PERTANYAAN ]
# Apa perbedaan hyperparameters dan parameters?

Hyperparameter: Diatur sebelum training, digunakan untuk mengontrol proses training.

Parameter: Dipelajari selama training, digunakan untuk prediksi.


In [ ]:
# TODO: Tentukan hyperparameters
epochs = 50
num_layers = 3
num_neurons = 128
learning_rate = 0.001

In [ ]:
# TODO: Tentukan besaran input untuk model
num_inputs = x_train.shape[1]

model = Net(in_features=num_inputs, num_layers=num_layers, num_neurons=num_neurons)
model = model.to(device)

In [ ]:
# Set the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [ ]:
# Check the number of parameters
print("Number of parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
print("Number of trainable parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Number of parameters: 3,969
Number of trainable parameters: 3,969


In [ ]:
# [ PERTANYAAN ]
# Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?

Semua parameter dalam model dapat di-train dan nilainya akan diperbarui selama proses training, sehingga total "trainable parameters" sama dengan total keseluruhan parameter.

### Train the Model

In [ ]:
print("Start training ...")
for epoch in range(epochs):
    train_loss = 0.0
    model.train()

    for data, label in Train_dataset:
        data = data.to(device)
        label = label.squeeze()
        label = label.to(device)
        optimizer.zero_grad()
        output = model.forward(data.float())

        loss = criterion(output.squeeze(), label.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss = train_loss / len(Train_dataset.dataset)
    if(epoch % 10 == 0):
        print('  - Epoch: {} \tTraining_loss: {:.6f}'.format(epoch, train_loss))

Start training ...
  - Epoch: 0 	Training_loss: 0.001244
  - Epoch: 10 	Training_loss: 0.000020
  - Epoch: 20 	Training_loss: 0.000009
  - Epoch: 30 	Training_loss: 0.000007
  - Epoch: 40 	Training_loss: 0.000005


### Model ACCURACY should reach >= 95%

In [ ]:
# TODO: Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning
from sklearn.metrics import accuracy_score

model.eval()
y_pred_list = []
y_true_list = []

with torch.no_grad():
    for data, label in Test_dataset:
        data = data.to(device)
        label = label.squeeze()
        label = label.to(device)

        output = model.forward(data.float())
        pred = (output > 0.5).float()
        y_pred_list.extend(pred.cpu().numpy().flatten())
        y_true_list.extend(label.cpu().numpy().flatten())


accuracy = accuracy_score(y_true_list, y_pred_list)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9997


In [ ]:
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for data, labels in Test_dataset:
        labels = labels.squeeze()
        output = model.forward(data.float())
        output = output.squeeze(1)

        predictions = (output >= 0.5).float()
        correct_preds += (predictions == labels).sum().item()
        total_samples += labels.numel()

accuracy = correct_preds / total_samples
print("Model accuracy: {:.2f}%".format(accuracy*100))

Model accuracy: 99.97%


### Scoring
Total `#TODO` = 12
<br>Checklist:

- [✔️] Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
- [✔️] Hilangkan kolom ID
- [✔️] Tentukan X (features) dan Y (target), gunakan "data_gpu"
- [✔️] Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
- [✔️] Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
- [✔️] Aktifkan GPU (CUDA) sebagai device untuk training
- [✔️] Tentukan nilai batch
- [✔️] PERTANYAAN: Apa perbedaan hyperparameters dan parameters?
- [✔️] Tentukan hyperparameters
- [✔️] Tentukan besaran input untuk model
- [✔️] PERTANYAAN: Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?
- [✔️] Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning

### Additional readings
- N/A

### Copyright © 2024 Startup Campus, Indonesia
* Prepared by **Nicholas Dominic, M.Kom.** [(profile)](https://linkedin.com/in/nicholas-dominic)
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.